In [1]:
import pandas as pd

In [2]:
cd ..

/run/media/jeevan/Anime Drive/nlp-fall-2019-project-reddit_upvote-predictor


In [3]:
newdata = pd.read_csv('data/data.csv')
newdata

,comment_body,upvotes
0,Imagine how beautiful this is in colors.,158
1,Them cape points are on point!,57
2,Now there’s a Batman,46
3,"Say whatever you want, Snyder knows how to fil...",104
4,God that man has a gift for shooting some amaz...,47
...,...,...
161305,I’m still waiting for the 3rd act or so where ...,1
161306,"It's like 2 steps forward (aquaman, shazam!) 3...",1
161307,"Either would be fine, but Cassie's Batgirl cos...",1
161308,Huntress appears to be wearing the same mask i...,3


Some of the tweets were not saved in the right format (text was divided/only fetched some of the text). Hence we remove these rows from the data.

In [4]:
newdata['upvotes'] = pd.to_numeric(newdata['upvotes'], errors='coerce')
newdata.dropna()

,comment_body,upvotes
0,Imagine how beautiful this is in colors.,158.0
1,Them cape points are on point!,57.0
2,Now there’s a Batman,46.0
3,"Say whatever you want, Snyder knows how to fil...",104.0
4,God that man has a gift for shooting some amaz...,47.0
...,...,...
161305,I’m still waiting for the 3rd act or so where ...,1.0
161306,"It's like 2 steps forward (aquaman, shazam!) 3...",1.0
161307,"Either would be fine, but Cassie's Batgirl cos...",1.0
161308,Huntress appears to be wearing the same mask i...,3.0


In [5]:
newdata.describe()

,upvotes
count,161067.000000
mean,13.281088
std,46.051583
min,-240.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,2079.000000


In [6]:
#newdata = newdata[0:10000]
newdata.loc[:, 'comment_body'] = newdata['comment_body'].str.replace('([\“”¨«»®´·º½¾¿¡§£₤‘’\n\r>])', '').str.replace('\[\S+\]', '').str.replace('\(\S+\)', '').str.replace('[http]{4}\S+', '')
#newdata['comment_body'] = [data.replace(';', ':') for data in newdata['comment_body']]
newdata = newdata.dropna()
newdata['length'] = [len(x) for x in newdata['comment_body']]
newdata = newdata[newdata.length <= 512]
ids = [i for i in range(len(newdata))]
ups = list(newdata["upvotes"])

<ipython-input-6-2a095d2b9043>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['length'] = [len(x) for x in newdata['comment_body']]


In [7]:
newdata.describe()

,upvotes,length
count,153581.000000,153581.000000
mean,13.516978,112.127906
std,46.653778,102.862073
min,-240.000000,0.000000
25%,1.000000,38.000000
50%,3.000000,77.000000
75%,9.000000,152.000000
max,2079.000000,512.000000


Based on the summary statistics, we define four categories for upvotes, where x is the upvote count:

x < 0 : Poor (label: 0)

x >= 1 and x <= 3: Normal (label: 1)

x >= 4 and x <=9: Good (label: 2)

x >= 10: Best (label: 3)

Also notice that the statistics didn't change by much, even though we removed all comments with length more than 512 (~8k comments)

In [ ]:
newdata["label"] = pd.cut(newdata["upvotes"], bins=[-np.inf, 0., 3, 9, np.inf], labels=[0, 1, 2, 3])
newdata["text"] = newdata["comment_body"]
cols = ["label", "text"]
newdata = newdata[cols]

In [30]:
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

#import nltk
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
#nltk.download('punkt')

def mostcommon(given):
    giventext = " ".join(list(given['text']))
    #giventext = giventext.lower().split()
    tokens = word_tokenize(giventext)
    tokens = [x.lower() for x in tokens if x.lower() not in stop_words]
    tokens = [re.sub('[\.,?!/()&]', '', x) for x in tokens]
    tokens = [x for x in tokens if x != '']
    #Return top 10 words
    return Counter(tokens).most_common()[:10]

In [31]:
best = newdata[newdata.label == 3]
good = newdata[newdata.label == 2]
normal = newdata[newdata.label == 1]
poor = newdata[newdata.label == 0]

print(mostcommon(best))
print(mostcommon(good))
print(mostcommon(normal))
print(mostcommon(poor))

[("'s", 8416), ("n't", 5565), ('like', 5060), ('movie', 4442), ('would', 3139), ('batman', 2957), ('one', 2574), ('think', 2506), ('``', 2298), ("''", 2252)]
[("'s", 7416), ("n't", 5352), ('like', 4530), ('movie', 3917), ('would', 2860), ('batman', 2716), ('think', 2362), ('one', 2217), ('people', 2015), ('superman', 1969)]
[("'s", 15092), ("n't", 11908), ('like', 9654), ('movie', 8367), ('would', 6116), ('batman', 5864), ('think', 5103), ('one', 4706), ('good', 4177), ('see', 4172)]
[("'s", 4225), ("n't", 3787), ('like', 2856), ('movie', 2685), ('batman', 1714), ('would', 1503), ('people', 1411), ('one', 1269), ('good', 1232), ('think', 1221)]


In [ ]:
newdata = newdata.sample(frac=1).reset_index(drop=True)
splitby = int(0.8*len(newdata))
train_data = newdata[:splitby]
test_data = newdata[splitby:]
train_data = train_data.dropna()
test_data = test_data.dropna()
train_data

In [ ]:
train_data.to_csv("train.csv", index=False, columns=["label", "text"])
test_data.to_csv("test.csv", index=False, columns=["label", "text"])